In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import pandas as pd
import re
from datetime import datetime
import sys 

quintile_mapping = {1: 'F2', 2: 'F1', 3: 'C', 4: 'B1', 5: 'B2'}
quintile_mapping_thirds = {1: 'F', 2: 'C', 3: 'B'}

from Convert_to_Profiles_Functions import *
from Get_Events_Functions import *
from Steef_Functions import *

gauge_nums = range(0,1294)
time_period='Future'

def find_max_quintile (precip, n):
    # Difference with this to Huff curve function, is it doesnt normalise
    cumulative_rainfall, cumulative_rainfall_times = create_cumulative_event(precip)
    dimensionless_cumulative_rainfall, dimensionless_times =  create_dimensionless_event(cumulative_rainfall, cumulative_rainfall_times)
    interpolated_n_cumulative_rainfall, interpolated_n_times = interpolate_rainfall(dimensionless_cumulative_rainfall,n)
    interpolated_n_incremental_rainfall = create_incremental_event(interpolated_n_cumulative_rainfall)
    max_quintile_profile = find_part_with_most_rain(interpolated_n_incremental_rainfall, n)
    
    return max_quintile_profile 

In [10]:
ems = ['bb192', 'bb195', 'bb198', 'bb201', 'bb204','bb208' ,'bb211','bb219','bb222','bb225']

In [ ]:
for em in ems:
    print(em)
    base_fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/"
    if em == 'nimrod':
        profiles_fp = f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/NIMROD_30mins/WholeYear/"
    else:
        profiles_fp = f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/UKCP18_30mins/{time_period}/{em}/"

    with open(profiles_fp + "df.pkl", 'rb') as file:
        df =pickle.load(file)

    # Create version without nulls    
    df_withoutnulls = df[df['precip'].notnull()].copy()       

    df_withoutnulls[['dimensionless_cumulative_rainfall', 'dimensionless_cumulative_times']] = df_withoutnulls['precip'].apply(create_huff_curves).apply(pd.Series)
    df_withoutnulls[['interpolated_cumulative_rainfall', 'interpolated_times']] = df_withoutnulls['dimensionless_cumulative_rainfall'].apply(lambda x: interpolate_rainfall(x,12)).apply(pd.Series)
    df_withoutnulls[['irain_14vals', 'irain_times_14vals']] = df_withoutnulls['interpolated_cumulative_rainfall'].apply(lambda x: create_irain_profile(x, True)).apply(pd.Series)
    df_withoutnulls[['irain', 'irain_times']] = df_withoutnulls['interpolated_cumulative_rainfall'].apply(lambda x: create_irain_profile(x, False)).apply(pd.Series)

    #  Add quintile cats
    df_withoutnulls['max_quintile_molly'] = df_withoutnulls['precip'].apply(lambda x: find_max_quintile(x,5))
    df_withoutnulls['max_third_molly'] = df_withoutnulls['precip'].apply(lambda x: find_max_quintile(x,3))
    df_withoutnulls['max_quintile_steef'] = df_withoutnulls['precip'].apply(analyse_event)
    df_withoutnulls['max_quintile_raw_rain'] = df_withoutnulls['precip'].apply(lambda x: find_part_with_most_rain(x, 5))
    df_withoutnulls[['max_quintile_steef', 'irain_profile_12_Steef']] = df_withoutnulls['precip'].apply(lambda x: pd.Series(analyse_event(x)))

    # Add loading cats
    df_withoutnulls['Loading_profile_raw_rain'] = df_withoutnulls['max_quintile_raw_rain'].map(quintile_mapping)
    df_withoutnulls['Loading_profile_molly'] =df_withoutnulls['max_quintile_molly'].map(quintile_mapping)
    df_withoutnulls['Loading_profile_third_molly'] =df_withoutnulls['max_third_molly'].map(quintile_mapping_thirds)
    df_withoutnulls['Loading_profile_steef'] =df_withoutnulls['max_quintile_steef'].map(quintile_mapping)

    # Assuming df is your DataFrame and 'duration_column' is the column with the original labels
    duration_labels_old = ['0.25-2.10 hr', '2.10-6.45 hr', '6.45-19.25 hr', '19.25+ hr']
    duration_labels_new = ['1.5-5hr', '5-11.5hr', '11.5-22.5hr', '22.5+hr']

    # Create a dictionary to map old values to new ones
    duration_mapping = dict(zip(duration_labels_old, duration_labels_new))

    # Replace the values in the specified column
    df_withoutnulls['DurationRange_personalised_allems'] = df_withoutnulls['DurationRange_personalised_allems'].replace(duration_mapping)

    # Save
    # Assuming df is your DataFrame and 'duration_column' is the column with the original labels
    duration_labels_old = ['0.25-2.10 hr', '2.10-6.45 hr', '6.45-19.25 hr', '19.25+ hr']
    duration_labels_new = ['1.5-5hr', '5-11.5hr', '11.5-22.5hr', '22.5+hr']

    # Create a dictionary to map old values to new ones
    duration_mapping = dict(zip(duration_labels_old, duration_labels_new))

    # Replace the values in the specified column
    df_withoutnulls['DurationRange_personalised_allems'] = df_withoutnulls['DurationRange_personalised_allems'].replace(duration_mapping)

    # Save
    with open(profiles_fp + "df_nonulls.pkl", 'wb') as file:
        pickle.dump(df_withoutnulls, file)    

    with open(profiles_fp + "df.pkl", 'wb') as file:
        pickle.dump(df, file)  

bb195
bb198
bb201
bb204
bb208
bb211
bb219
bb222


In [8]:
# time_period='nimrod'
# ems = ['nimrod']
# for em in ems:
#     print(em)
#     base_fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/"
#     if em == 'nimrod':
#         profiles_fp = f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/NIMROD_30mins/WholeYear/"
#     else:
#         profiles_fp = f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/UKCP18_30mins/{time_period}/{em}/"

#     with open(profiles_fp + "df.pkl", 'rb') as file:
#         df =pickle.load(file)

#     # Create version without nulls    
#     df_withoutnulls = df[df['precip'].notnull()].copy()       

#     df_withoutnulls[['dimensionless_cumulative_rainfall', 'dimensionless_cumulative_times']] = df_withoutnulls['precip'].apply(create_huff_curves).apply(pd.Series)
#     df_withoutnulls[['interpolated_cumulative_rainfall', 'interpolated_times']] = df_withoutnulls['dimensionless_cumulative_rainfall'].apply(lambda x: interpolate_rainfall(x,12)).apply(pd.Series)
#     df_withoutnulls[['irain_14vals', 'irain_times_14vals']] = df_withoutnulls['interpolated_cumulative_rainfall'].apply(lambda x: create_irain_profile(x, True)).apply(pd.Series)
#     df_withoutnulls[['irain', 'irain_times']] = df_withoutnulls['interpolated_cumulative_rainfall'].apply(lambda x: create_irain_profile(x, False)).apply(pd.Series)

#     #  Add quintile cats
#     df_withoutnulls['max_quintile_molly'] = df_withoutnulls['precip'].apply(lambda x: find_max_quintile(x,5))
#     df_withoutnulls['max_third_molly'] = df_withoutnulls['precip'].apply(lambda x: find_max_quintile(x,3))
#     df_withoutnulls['max_quintile_steef'] = df_withoutnulls['precip'].apply(analyse_event)
#     df_withoutnulls['max_quintile_raw_rain'] = df_withoutnulls['precip'].apply(lambda x: find_part_with_most_rain(x, 5))
#     df_withoutnulls[['max_quintile_steef', 'irain_profile_12_Steef']] = df_withoutnulls['precip'].apply(lambda x: pd.Series(analyse_event(x)))

#     # Add loading cats
#     df_withoutnulls['Loading_profile_raw_rain'] = df_withoutnulls['max_quintile_raw_rain'].map(quintile_mapping)
#     df_withoutnulls['Loading_profile_molly'] =df_withoutnulls['max_quintile_molly'].map(quintile_mapping)
#     df_withoutnulls['Loading_profile_third_molly'] =df_withoutnulls['max_third_molly'].map(quintile_mapping_thirds)
#     df_withoutnulls['Loading_profile_steef'] =df_withoutnulls['max_quintile_steef'].map(quintile_mapping)

#     # Save
#     with open(profiles_fp + "df_nonulls.pkl", 'wb') as file:
#         pickle.dump(df_withoutnulls, file)    

#     with open(profiles_fp + "df.pkl", 'wb') as file:
#         pickle.dump(df, file)  

In [9]:
for em in ['nimrod_5mins']:
    print(em)
    base_fp = f"/nfs/a161/gy17m2a/PhD/ProcessedData/"
    if em == 'nimrod_5mins':
        profiles_fp = f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/NIMROD_5mins/WholeYear/1km_filtered_100/"    
    elif em == 'nimrod_30mins':
        profiles_fp = f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/NIMROD_30mins/WholeYear/"
    else:
        profiles_fp = f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/UKCP18_30mins/{time_period}/{em}/"

    with open(profiles_fp + "df.pkl", 'rb') as file:
        df =pickle.load(file)

    # Create version without nulls    
    df_withoutnulls = df[df['precip'].notnull()].copy()       

    df_withoutnulls[['dimensionless_cumulative_rainfall', 'dimensionless_cumulative_times']] = df_withoutnulls['precip'].apply(create_huff_curves).apply(pd.Series)
    df_withoutnulls[['interpolated_cumulative_rainfall', 'interpolated_times']] = df_withoutnulls['dimensionless_cumulative_rainfall'].apply(lambda x: interpolate_rainfall(x,12)).apply(pd.Series)
    df_withoutnulls[['irain_14vals', 'irain_times_14vals']] = df_withoutnulls['interpolated_cumulative_rainfall'].apply(lambda x: create_irain_profile(x, True)).apply(pd.Series)
    df_withoutnulls[['irain', 'irain_times']] = df_withoutnulls['interpolated_cumulative_rainfall'].apply(lambda x: create_irain_profile(x, False)).apply(pd.Series)

    #  Add quintile cats
    df_withoutnulls['max_quintile_molly'] = df_withoutnulls['precip'].apply(lambda x: find_max_quintile(x,5))
    df_withoutnulls['max_third_molly'] = df_withoutnulls['precip'].apply(lambda x: find_max_quintile(x,3))
    df_withoutnulls['max_quintile_steef'] = df_withoutnulls['precip'].apply(analyse_event)
    df_withoutnulls['max_quintile_raw_rain'] = df_withoutnulls['precip'].apply(lambda x: find_part_with_most_rain(x, 5))
    df_withoutnulls[['max_quintile_steef', 'irain_profile_12_Steef']] = df_withoutnulls['precip'].apply(lambda x: pd.Series(analyse_event(x)))

    # Add loading cats
    df_withoutnulls['Loading_profile_raw_rain'] = df_withoutnulls['max_quintile_raw_rain'].map(quintile_mapping)
    df_withoutnulls['Loading_profile_molly'] =df_withoutnulls['max_quintile_molly'].map(quintile_mapping)
    df_withoutnulls['Loading_profile_third_molly'] =df_withoutnulls['max_third_molly'].map(quintile_mapping_thirds)
    df_withoutnulls['Loading_profile_steef'] =df_withoutnulls['max_quintile_steef'].map(quintile_mapping)

    # Assuming df is your DataFrame and 'duration_column' is the column with the original labels
    duration_labels_old = ['0.25-2.10 hr', '2.10-6.45 hr', '6.45-19.25 hr', '19.25+ hr']
    duration_labels_new = ['1.5-5hr', '5-11.5hr', '11.5-22.5hr', '22.5+hr']

    # Create a dictionary to map old values to new ones
    duration_mapping = dict(zip(duration_labels_old, duration_labels_new))

    # Replace the values in the specified column
    df_withoutnulls['DurationRange_personalised_allems'] = df_withoutnulls['DurationRange_personalised_allems'].replace(duration_mapping)

    # Save
    with open(profiles_fp + "df_nonulls.pkl", 'wb') as file:
        pickle.dump(df_withoutnulls, file)    

    with open(profiles_fp + "df.pkl", 'wb') as file:
        pickle.dump(df, file)  

nimrod_5mins
